In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier


# Загрузка данных

In [ ]:
train_df = pd.read_csv("/kaggle/input/titanic/train.csv")
train_df.sample(10)

In [ ]:
train_df.info()

In [ ]:
test_df= pd.read_csv("/kaggle/input/titanic/test.csv")
test_df.head()

In [ ]:
test_df.shape

Приведем столбец Fare к типу int64.

In [ ]:
train_df['Fare']= train_df['Fare'].astype('int')

Проверим данные на пропуски.

In [ ]:
train_df.isnull().sum()

Пропуски присутсвуют, нужно определиться, как их заполнить. 

Взглянем на корреляцию данных между столбцами.

In [ ]:
train_df.corr()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(train_df.corr('pearson'), vmin=-1, vmax=1, cmap='coolwarm', annot=True, square=True)
plt.show()

Из графиков корреляции видим, что высоких зависимостей между колонками - нет.

# Исследовательский анализ данных

In [ ]:
for cols in ['Sex','Embarked','Pclass', 'SibSp', 'Parch']:
    print(cols)
    sns.countplot(data=train_df, x=cols, hue='Survived')
    plt.show()
    print("")

Из графиков видим следующее: 
1. В основном, выжили женщины - мужчины пропускали их на спасательные шлюпки в первую очередь. 
2. Большая часть пассажиров отправлялись из порта города Southampton, от того крупная часть потерь среди тех, кто садился на корабль из этого города. 
3. Значительная часть пассажиров приобретали билеты 3 (эконом) класса - среди именно этого класса билетов много потерь. Также заметно, что большая часть пассажиров из 1 (люкс) класса  - выжили. 
4. Графики количества родственников пассажиров на борту показывают, что большинство пассажиров путешествовали в одиночку, однако, как показала ситуация, больше шансов на выживание было у людей с 1 родственником на борту.


Взглянем на пропуски в колонке Embarked.

In [ ]:
nan_rows_Embarked = train_df[train_df['Embarked'].isna()]
nan_rows_Embarked.head()

In [ ]:
selected_rows = train_df[(train_df['Pclass'] == 1) & (train_df['Survived'] == 1) & (train_df['Sex'] == 'female') & (train_df['Fare']< 82) & (train_df['Fare']> 78)]
selected_rows.head(20)

Заполним колонку подходящим значением.

In [ ]:
median_embarked = train_df['Embarked'].mode()[0]
train_df['Embarked'].fillna(median_embarked, inplace=True)

In [ ]:
selected_rows = train_df[(train_df['Pclass'] == 1) & (train_df['Survived'] == 1) & (train_df['Sex'] == 'female') & (train_df['Fare']< 82) & (train_df['Fare']> 78)]
selected_rows.head(20)

Удалим лишние колонки.

In [ ]:
train_df.drop(['Name','PassengerId','Ticket'], axis= 1, inplace= True)

Посмотрим на распределение возраста пассажиров.

In [ ]:
sns.displot(train_df['Age'], kde= True)

Правильнее всего будет заполнить пропуски в колонке Age - медианным значением, так как на него не влияют выбросы значений, следовательно, усредненный таким образом возраст подойдет лучше.

In [ ]:
median_age = train_df['Age'].median()
train_df['Age'].fillna(median_age, inplace=True)

In [ ]:
print(train_df['Age'].isna())

Создадим новый столбец, показывающий, была ли каюта у пассажира и высчитаем шанс на выживание у людей с каютой, а также без нее.

In [ ]:

train_df['has_cabin'] = train_df['Cabin'].notnull().astype(int)



survival_rates = train_df.groupby('has_cabin')['Survived'].mean()


print(survival_rates)

Видим, что люди жившие в каютах имели шанс на выживание в катастрофе - 67%.

In [ ]:
print(train_df[['Cabin', 'has_cabin']].head())

In [ ]:

num_zeros = (train_df['has_cabin'] == 0).sum()


print(num_zeros)

Судя по тому, что у нас достаточно много пропусков в столбце наличия каюты - оставим бинарный столбец, а оригинальный столбец с номером - удалим.

In [ ]:
train_df = train_df.drop('Cabin', axis=1)
train_df.head()

In [ ]:
train_df.info()

# Подготовка данных к обучению

Так как на графике распределения возраста была явная скошенность - применим StandartScaler и OneHotEncoder к категориальным и количественным данным.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age', 'Fare']), 
        ('cat', OneHotEncoder(sparse_output= False, drop='first'), ['Sex', 'Embarked']) 
    ])

In [ ]:
X= preprocessor.fit_transform(train_df)
X.shape

Получили данные типа array - преобразуем их в новый датасет.

In [ ]:
X_transformed = np.hstack((train_df, X))

In [ ]:
final_df= pd.DataFrame(columns=['survived','pclass','Sex','age','sibsp','parch','fare','emarked','cabin','nage','nfare','sex','em1','em2'],data= X_transformed)
final_df.head()

После преобразования данных можно удалить столбцы 'Sex', 'age' и 'fare', так как появились аналогичные колонки с названиями 'nage', 'nfare', 'sex', 'em1', 'em2'.

In [ ]:
main_df = final_df[['pclass','sibsp','parch','cabin','nage','nfare','sex','em1','em2']]
main_df = main_df.astype(int)
target = final_df['survived']
target = target.astype(int)

In [ ]:
main_df.info()

# Обучение моделей

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(main_df, target, test_size = 0.1, random_state=1234)

In [ ]:

lr = LogisticRegression()

score_lr = cross_val_score(lr,x_train, y_train, cv= 20)
print(score_lr.mean())

score_lr = score_lr.mean()

# Random Forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=10, random_state= 1234)


    
score_rfc = cross_val_score(rfc,x_train, y_train, cv= 20)
print(score_rfc.mean())

score_rfc = score_rfc.mean()

# Decision Tree

In [ ]:
dtc = DecisionTreeClassifier(random_state=1234)



score_dtc = cross_val_score(dtc,x_train, y_train, cv= 20)
print(score_dtc.mean())

score_dtc = score_dtc.mean()

# K-Neighbours

In [ ]:
knn= KNeighborsClassifier()



score_knn = cross_val_score(knn ,x_train, y_train, cv= 20)
print(score_knn.mean())

score_knn = score_knn.mean()

# Сравнение моделей

In [ ]:
models = ['Logistic Regression', 'Random Forest','Decision Tree', 'K-Neighbours']
score = [score_lr, score_rfc, score_dtc, score_knn]

model_comp = pd.DataFrame({'Model': models,'Score': score})

display(model_comp)

Модель Random Forest показала наиболее качественные результаты. Лучший результат: 0.807.

In [ ]:
from sklearn.ensemble import RandomForestClassifier



y = train_df["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_df[features])
X_test = pd.get_dummies(test_df[features])

model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=1234)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)

# Заключение

Многие вещи увеличили бы шансы на выживание пассажиров, но далеко не все можно было предусмотреть и изменить. Женщины и дети спаслись с большей вероятностью, пассажиры первого класса выжили с большей вероятностью, чем пассажиры второго и третьего классов. Однако, если вы - мужчина на Титанике, то ваш шанс на выживание больше, если на корабль вы сели из Cherbourgh. Также, путешествие, как минимум, с 1 родственником также увеличивало вероятность выживания.